# Gebundene Eigenzustände des endlichen Potentialtops

Siehe Wiedemann/Ingold: *Numerische Physik mit Python*, Springer-Spektrum 2024, ISBN 978-3-662-69566-1

---

In diesem Jupyter-Notebook werden die gebundenen Zustände für einen endlich tiefen Potentialtopf untersucht. Dazu werden die Eigenzustände der zeitunabhängigen Schrödingergleichung gesucht, die in dimensionsloser Form durch

$$-\frac{\text{d}^2}{\text{d}x^2}\psi(x) + \alpha^2 v(x)\psi(x) = E \psi(x)$$

gegeben ist. Der Parameter $\alpha^2$ gibt die Tiefe des Potentials an. Das dimensionslose Potential lautet

$$v(x) = \begin{cases} -1 & \text{für $\vert x\vert < 1$}\\ 0 & \text{sonst}\end{cases}$$

Im ersten Teil des Jupyter-Notebooks sehen wir uns die Lösungen der Schrödingergleichung für beliebige negative Energien an. Dabei wird sich zeigen, dass sich nur für bestimmte Werte der Energie normierbare Zustände ergeben. Anschließend werden wir uns die analytisch erhaltenen Eigenwertbedingungen ansehen und diese durch eine Nullstellensuche lösen. Damit sind wir dann auch in der Lage, die zugehörigen Eigenfunktionen graphisch darzustellen. Im letzten Teil betrachten wir noch den Übergang zum Deltapotential.

## Importanweisungen

In [ ]:
from math import cos, exp, floor, pi, sin, sqrt, tan
import numpy as np
from scipy import optimize
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt

plt.style.use("numphyspy.style")

## Lösungen der Schrödingergleichung als Funktion der Energie

Zunächst sehen wir uns an, was passiert, wenn wir die Schrödingergleichung für eine beliebige negative Energie lösen. Da die Lösungen in den Teilbereichen konstanten Potentials analytisch bekannt sind, verzichten wir hier auf eine numerische Lösung. Wir müssen jedoch dafür sorgen, dass die Anschlussbedingungen für die Wellenfunktion an den Sprungstellen des Potentials erfüllt sind. In der graphischen Darstellung werden wir feststellen, dass die Lösungen im Unendlichen normalerweise divergieren und somit nicht normierbar sind. Allerdings wechselt die Wellenfunktion im Unendlichen für bestimmte Energien ihr Vorzeichen. Auch wenn diese Energien mit dem Schieberegler schwer exakt einzustellen sind, liegen dort die physikalisch relevanten Lösungen der Schrödingergleichung.

Mit Hilfe der Bedienelemente lassen sich die folgenden Parameter einstellen:
- `symmetry`: Symmetrie des Zustands (symmetrisch oder antisymmetrisch)
- `energy`: Energie in Einheiten von $\alpha^2$. Der Boden des Potentialtopfs liegt in diesen Einheiten bei -1.

In [ ]:
widget_dict = {"symmetry":
               widgets.RadioButtons(
                   options=["symmetrisch",
                            "antisymmetrisch"],
                   description="Symmetrie"),
               "energy":
               widgets.FloatSlider(
                   value=-0.5, min=-1, max=-1e-3,
                   step=0.001,
                   description=r"$E/\alpha^2$",
                   readout_format=".3f")
               }

@interact(**widget_dict)
def plot_state(symmetry, energy):
    alpha = 5.5
    energy = energy * alpha**2
    x_max = 3
    x_I = np.linspace(0, 1, 100)
    x_II = np.linspace(1, x_max, 100)
    k = sqrt(alpha**2+energy)
    kappa = sqrt(-energy)
    if symmetry == "symmetrisch":
        factor = 1
        psi_I = np.cos(k*x_I)
        b = 0.5*exp(kappa)*(cos(k)+k/kappa*sin(k))
        c = 0.5*exp(-kappa)*(cos(k)-k/kappa*sin(k))
    else:
        factor = -1
        psi_I = np.sin(k*x_I)
        b = 0.5*exp(kappa)*(sin(k)-k/kappa*cos(k))
        c = 0.5*exp(-kappa)*(sin(k)+k/kappa*cos(k))
    psi_II = b*np.exp(-kappa*x_II) + c*np.exp(kappa*x_II)

    fig, ax = plt.subplots()
    ax.plot(x_I, psi_I, "blue")
    ax.plot(-x_I, factor*psi_I, "blue")
    ax.plot(x_II, psi_II, "blue")
    ax.plot(-x_II, factor*psi_II, "blue")
    ax.vlines([-1, 1], [-2, -2], [2, 2],
              linestyle="dashed", color="black")
    ax.set_xlabel("$x$")
    ax.set_ylabel(r"$\psi(x)$")
    ax.set_ylim(-2, 2)

## Eigenenergien und Eigenzustände aus den analytischen Eigenwertbedingungen

### Darstellung der Eigenwertbedingungen

Aus der Normierbarkeitsbedingung für die Eigenzustände ergeben sich Eigenwertbedingungen für die Wellenzahl $k$, die mit der Energie über

$$k = \sqrt{\alpha^2+E}$$

zusammenhängt. Die Eigenwertbedingungen lauten für symmetrische Eigenzustände

$$\sqrt{\alpha^2-k^2} = k\tan(k)$$

und für antisymmetrische Eigenzustände

$$\sqrt{\alpha^2-k^2} = -k\cot(k)\,.$$

Im Folgenden stellen wir die linke sowie die beiden rechten Seiten dieser Eigenwertbedingungen dar. Jeder Schnittpunkt der rot dargestellten linken Seite mit der blau dargestellten rechten Seite der ersten Eigenwertbedingung ergibt die Energie eines symmetrischen Eigenzustands. In jedem Intervall
$[n \pi/\alpha, (n+1/2) \pi/\alpha]$ liegt genau ein solcher Schnittpunkt. Umgekehrt entspricht jeder Schnittpunkt der roten und der grünen Kurve der Energie eines antisymmetrischen Eigenzustands. Diese Schnittpunkte liegen jeweils in den Intervallen $[(n+1/2)\pi/\alpha, (n+1) \pi/\alpha]$.

Mit Hilfe des Schiebereglers lässt sich der folgende Parameter einstellen:
- `alpha`: Wurzel aus der dimensionslosen Potentialtopftiefe

In [ ]:
widget_dict = {"alpha":
               widgets.FloatSlider(
                   value=5.5, min=0.5, max=10, step=0.1,
                   description=r"$\alpha$")
               }

@interact(**widget_dict)
def plot_ev_condition(alpha):
    k = np.linspace(1e-5, alpha, 300)
    lhs = np.sqrt(alpha**2-k**2)
    rhs_symm = k*np.tan(k)
    rhs_antisymm = -k/np.tan(k)

    fig, ax = plt.subplots()
    ax.plot(k, lhs, label="linke Seite", color="red")
    ax.plot(k, rhs_symm, label="rechte Seite, symm.",
            color="blue")
    ax.plot(k, rhs_antisymm,
            label="rechte Seite, antisymm.", color="green")
    ax.set_xlabel("$k$")
    ax.legend(loc="lower left")
    ax.set_ylim(-6, 6)

### Berechnung der Eigenenergien

Die Eigenenergien können wir nun mit Hilfe einer Nullstellensuche unter Verwendung der Funktion `optimize.brentq` aus der SciPy-Bibliothek bestimmen. Hierbei hilft uns, dass wir die Intervalle kennen, in denen Nullstellen zu finden sind. In der Funktion `eigenvalue_condition` nutzen wir aus, dass wir die beiden Eigenwertbedingungen in gleicher Weise formulieren können, indem wir den Parameter `symm` mit den Werten +1 und -1 für symmetrische bzw. antisymmetrische Eigenzustände einführen. Da wir die Anzahl der Eigenenergien nicht kennen, gibt die Generatorfunktion `get_eigenenergies` nacheinander die jeweiligen Werte der Eigenenergien zurück.

In [ ]:
def eigenvalue_condition(k, symm, alpha):
    return sqrt(alpha**2-k**2) - symm*k*tan(k)**symm

In [ ]:
def get_eigenenergies(alpha):
    a = 0
    b = min(alpha, pi/2)
    symm = 1
    eps = 1e-12
    while alpha > a:
        k = optimize.brentq(eigenvalue_condition,
                            a+eps, b-eps, (symm, alpha,))
        yield k**2-alpha**2
        a, b = b, min(alpha, b+pi/2)
        symm = -symm

### Berechnung der Eigenzustände

Für einen Potentialtopf, dessen Tiefe durch den Parameter `alpha` bestimmt ist, wird für den bereits erwähnten Symmetrieparameter `symm` und die Energie `energy` der zugehörige Eigenzustand auf einem Gitter mit `n_max` Punkten zwischen `-x_max` und `x_max` berechnet. Dabei werden die analytisch bekannten Ausdrücke für die Eigenfunktionen verwendet. Auf eine Normierung wird hier verzichtet.

In [ ]:
def eigenstate(symm, alpha, energy, n_max, x_max):
    if symm == 1:
        f_osc = np.cos
    else:
        f_osc = np.sin
    x_values = np.linspace(-x_max, x_max, 2*n_max+1)
    psi_values = np.empty(2*n_max+1)
    n_1 = floor(n_max/x_max)
    k = sqrt(alpha**2 + energy)
    kappa = sqrt(-energy)
    x_I = x_values[n_max-n_1:n_max+n_1+1]
    psi_values[n_max-n_1:n_max+n_1+1] = f_osc(k*x_I)
    x_II = x_values[:n_max-n_1]
    a_II = symm * f_osc(k) * exp(kappa)
    psi_II = a_II*np.exp(kappa*x_II)
    psi_values[:n_max-n_1] = psi_II
    psi_values[n_max+n_1+1:] = symm * psi_II[::-1]
    return x_values, psi_values

### Implementierung des Bedienelements und graphische Darstellung der Eigenzustände

Mit dem Schieberegler lässt sich der folgende Parameter einstellen:
- `alpha`: Wurzel aus der dimensionslosen Potentialtopftiefe

In der graphischen Darstellung der Eigenfunktionen wird die Nulllinie um die jeweilige Eigenenergie verschoben. Außerdem wird zur besseren Interpretation der Eigenfunktionen der Potentialverlauf dargestellt. Durch Variation des Parameters `alpha` verändert sich die Anzahl der Eigenzustände.

In [ ]:
widget_dict = {"alpha":
               widgets.FloatSlider(
                   value=5.5, min=0.5, max=10, step=0.5,
                   description=r"$\alpha$")
               }

@interact(**widget_dict)
def plot_eigenstates(alpha):
    n_max = 200
    x_max = 3

    fig, ax = plt.subplots()
    ax.plot([-x_max, -1, -1, 1, 1, x_max],
            [0, 0, -alpha**2, -alpha**2, 0, 0], "black")
    symm = 1
    for energy in get_eigenenergies(alpha):
        x_values, psi_values = eigenstate(
            symm, alpha, energy, n_max, x_max)
        ax.plot(x_values, 2*psi_values+energy)
        symm = -symm
    ax.set_xlabel("$x$")
    ax.set_ylabel(r"$\psi$")

## Übergang zum Delta-Potential

Indem man in unskalierten Einheiten die Breite $2 x_0$ des Potentialtopfs gegen null und dessen Tiefe $V_0$ gegen unendlich gehen lässt, wobei das Produkt $\eta$ dieser beiden Größen konstant gehalten wird, erhält man das Delta-Potential:

$$ V(x) = \eta \delta(x)\,.$$

Im Gegensatz zu den Eigenfunktionen des endlichen Potentialtopfs ist dessen Grundzustand an der Stelle $x=0$ nicht differenzierbar, sondern weist einen Knick auf. Links und rechts dieser Stelle ist die Wellenfunktion durch eine abfallende Exponentialfunktion gegeben. Da im Deltapotential nur ein einziger gebundener Zustand existiert, wird hier lediglich der Grundzustand als Funktion von $\alpha$ dargestellt, wobei uns das Verhalten für immer kleinere Werte von $\alpha$ interessiert. Da $\alpha$ hier variabel ist und es damit nicht mehr sinnvoll ist, Längen in Einheiten von $x_0$ zu nehmen, wählen wir für die Ordinate $\alpha^2 x$. Damit werden Längen in Einheiten von $\hbar^2/2mx_0V_0=\hbar^2/\eta m$ genommen.

Mit dem Schieberegler lässt sich der folgende Parameter einstellen:
- `alpha`: Wurzel aus der dimensionslosen Potentialtopftiefe

In [ ]:
widget_dict = {"alpha":
               widgets.FloatLogSlider(
                   value=1, min=-2, max=0, step=0.1,
                   description=r"$\alpha$")
               }

@interact(**widget_dict)
def plot_groundstate(alpha):
    x_0 = alpha**2 / 2
    n_max = 200
    x_max = 5
    x_scaled_max = x_max / alpha**2
    groundstate_energy = next(get_eigenenergies(alpha))
    x_values, psi_values = eigenstate(
        1, alpha, groundstate_energy, n_max, x_scaled_max)
    x_values = x_values * alpha**2

    fig, ax = plt.subplots()
    ax.plot(x_values, psi_values)
    ax.set_xlabel(r"$\alpha^2 x$")
    ax.set_ylabel(r"$\psi$")